In [15]:
import numpy as np
from pathlib import Path
import tifffile
from tqdm import tqdm

# Constants
FULL_WELL_CAPACITY = 11000
QUANTUM_EFFICIENCY = 0.69

def normalize_image(image):
    """Normalize image to 0-1 range preserving dynamic range"""
    img_min = image.min()
    img_max = image.max()
    if img_max == img_min:
        return np.zeros_like(image)
    return (image - img_min) / (img_max - img_min)

def calculate_rms(image):
    """Calculate RMS value of an image"""
    return np.sqrt(np.mean(np.square(image)))

# Get all matching tif files
file_pattern = "/mnt/d/hyper-scope/data/interim/capillary/imgs/image_20240730_*.tif"
tif_files = list(Path("/").glob(file_pattern[1:]))  # Remove leading slash for Path

if not tif_files:
    print(f"No files found matching pattern: {file_pattern}")
    exit(1)

# Process each image
rms_values = []
for tif_file in tqdm(tif_files):
    try:
        # Load image
        image = tifffile.imread(str(tif_file))
        
        # Normalize image
        normalized_image = normalize_image(image)
        
        # Calculate RMS
        rms = calculate_rms(normalized_image[normalized_image > 0])
        rms_values.append(rms)
        
    except Exception as e:
        print(f"Error processing {tif_file}: {e}")
        continue

if not rms_values:
    print("No valid RMS values calculated")
    exit(1)

# Calculate average RMS
avg_rms = np.mean(rms_values)

# Calculate final result in electrons
electrons = avg_rms * FULL_WELL_CAPACITY * QUANTUM_EFFICIENCY

print(f"Number of files processed: {len(tif_files)}")
print(f"Average RMS: {avg_rms:.6f}")
print(f"Final result in electrons: {electrons:.2f}")

 82%|████████▏ | 3184/3879 [01:02<00:13, 52.38it/s]

Error processing /mnt/d/hyper-scope/data/interim/capillary/imgs/image_20240730_135141_642277.tif: failed to read 2016000 bytes, got 63360


100%|██████████| 3879/3879 [01:15<00:00, 51.07it/s]

Number of files processed: 3879
Average RMS: 0.033226
Final result in electrons: 252.18


In [19]:
h = 6.626e-34 # (Planck's constant)
c = 3 * 10e8 # (speed of light in m/s)
l = 515 # wavelength
exp_t = 10e-3 # 10ms
photon_flux = electrons / exp_t / (3.45 * 1e-6)
E = (photon_flux * h * c) / l # power
lux = E * 683 # V(λ)
lux

1.9270126047301897e-14